In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
splits = [10,20,30]
num_classes = len(splits)+1

In [3]:
from dataset import YouTubeDataset
dataset = YouTubeDataset(splits)

0 - T001: 0.0003380775451660156 seconds.
1 - T003: 0.0006480216979980469 seconds.
2 - T004: 0.00021409988403320312 seconds.
3 - T006: 0.00016427040100097656 seconds.
4 - T007: 0.00020503997802734375 seconds.
5 - T009: 0.00019931793212890625 seconds.
6 - T011: 0.0001380443572998047 seconds.
7 - T013: 0.00023293495178222656 seconds.
8 - T015: 0.00019097328186035156 seconds.
9 - T016: 0.00016999244689941406 seconds.
10 - T017: 0.0002079010009765625 seconds.
11 - T018: 0.00018715858459472656 seconds.
12 - T020: 0.0001647472381591797 seconds.
13 - T021: 0.0001590251922607422 seconds.
14 - T022: 0.0001919269561767578 seconds.
15 - T023: 0.00018095970153808594 seconds.
16 - T024: 0.00018906593322753906 seconds.
17 - T027: 0.00018405914306640625 seconds.
18 - T028: 0.00015592575073242188 seconds.
19 - T031: 0.00010919570922851562 seconds.
20 - T033: 0.00015592575073242188 seconds.
21 - T034: 0.0001709461212158203 seconds.
22 - T035: 0.00013327598571777344 seconds.
23 - T036: 0.0001139640808105

In [5]:
from models.single_modality_classifiers import BagOfWordsClassifier
model = BagOfWordsClassifier(num_classes)

In [6]:
import torch
import math
import os
import time
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from inference import eval, get_scores
from torch.nn.functional import cross_entropy

def train_model(model, dataset, train_val_split = None):
    if not train_val_split:
      train_ids = [i for i in range(len(dataset))]
      val_ids = None
    else:
      train_ids, val_ids = train_val_split
    
    X_train, y_train = dataset.word_matrix[train_ids], dataset.label[train_ids].numpy()
    X_val, y_val = dataset.word_matrix[val_ids], dataset.label[val_ids].numpy()
    
    priors = np.zeros((model.num_classes,))
    for y in y_train:
      priors[y] += 1
    priors /= priors.sum()
    print(priors)

    model.reset(priors)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    train_accuracy, train_precision, train_recall, train_f1 = get_scores(y_train, y_pred, model.num_classes) # This is an aggregated result due to GPU size limit
    print(f"    Training Set - accuracy: {train_accuracy:.2f}, precision: {train_precision:.2f}, recall: {train_recall:.2f}, f1-score: {train_f1:.2f},")
    val_accuracy, val_precision, val_recall, val_f1 = get_scores(y_val, y_pred_val, model.num_classes)
    print(f"    Validation Set - accuracy: {val_accuracy:.2f}, precision: {val_precision:.2f}, recall: {val_recall:.2f}, f1-score: {val_f1:.2f},")

## 5-fold CV

In [10]:
from sklearn.model_selection import KFold
import torch

def train_model_cv5(model, dataset):
    loss_hist = []
    kf = KFold(n_splits=5)
    cnt = 1
    for train_index, val_index in kf.split(dataset):
        print("Fold "+str(cnt)+" (val", val_index[0],"-",str(val_index[-1])+")")
        loss_hist_fold = train_model(model, dataset=dataset, train_val_split=(train_index, val_index))
        loss_hist.append(loss_hist_fold)
        cnt += 1
    return loss_hist

In [11]:
lost_hist_folds = train_model_cv5(model, dataset)

Fold 1 (val 0 - 42)
[0.47619048 0.33928571 0.125      0.05952381]
    Training Set - accuracy: 1.00, precision: 1.00, recall: 1.00, f1-score: 1.00,
    Validation Set - accuracy: 0.65, precision: 0.28, recall: 0.37, f1-score: 0.29,
Fold 2 (val 43 - 84)
[0.5147929  0.30769231 0.1183432  0.0591716 ]
    Training Set - accuracy: 1.00, precision: 1.00, recall: 1.00, f1-score: 1.00,
    Validation Set - accuracy: 0.62, precision: 0.17, recall: 0.22, f1-score: 0.19,
Fold 3 (val 85 - 126)
[0.59171598 0.27218935 0.10059172 0.03550296]
    Training Set - accuracy: 1.00, precision: 1.00, recall: 1.00, f1-score: 1.00,
    Validation Set - accuracy: 0.45, precision: 0.23, recall: 0.30, f1-score: 0.25,
Fold 4 (val 127 - 168)
[0.5739645  0.29585799 0.07692308 0.05325444]
    Training Set - accuracy: 0.99, precision: 1.00, recall: 1.00, f1-score: 1.00,
    Validation Set - accuracy: 0.29, precision: 0.12, recall: 0.17, f1-score: 0.14,
Fold 5 (val 169 - 210)
[0.59171598 0.25443787 0.10059172 0.0532544

In [14]:
from dataset import YouTubeDataset
splits = [10,20]
num_classes = len(splits)+1
dataset = YouTubeDataset(splits)
model = BagOfWordsClassifier(num_classes)

hist = train_model_cv5(model, dataset)

0 - T001: 0.0003981590270996094 seconds.
1 - T003: 0.0006840229034423828 seconds.
2 - T004: 0.0003693103790283203 seconds.
3 - T006: 0.00025916099548339844 seconds.
4 - T007: 0.00023102760314941406 seconds.
5 - T009: 0.00020885467529296875 seconds.
6 - T011: 0.000148773193359375 seconds.
7 - T013: 0.000209808349609375 seconds.
8 - T015: 0.0001919269561767578 seconds.
9 - T016: 0.00020003318786621094 seconds.
10 - T017: 0.00019621849060058594 seconds.
11 - T018: 0.00019097328186035156 seconds.
12 - T020: 0.0001857280731201172 seconds.
13 - T021: 0.0001862049102783203 seconds.
14 - T022: 0.00018143653869628906 seconds.
15 - T023: 0.000202178955078125 seconds.
16 - T024: 0.0001900196075439453 seconds.
17 - T027: 0.00019621849060058594 seconds.
18 - T028: 0.00017070770263671875 seconds.
19 - T031: 0.00014734268188476562 seconds.
20 - T033: 0.0001709461212158203 seconds.
21 - T034: 0.00018215179443359375 seconds.
22 - T035: 0.00013303756713867188 seconds.
23 - T036: 0.00011587142944335938 s

In [13]:
from dataset import YouTubeDataset
splits = [10]
num_classes = len(splits)+1
dataset = YouTubeDataset(splits)
model = BagOfWordsClassifier(num_classes)

hist = train_model_cv5(model, dataset)

0 - T001: 0.0003509521484375 seconds.
1 - T003: 0.0004286766052246094 seconds.
2 - T004: 0.00021314620971679688 seconds.
3 - T006: 0.0001342296600341797 seconds.
4 - T007: 0.00020694732666015625 seconds.
5 - T009: 0.00018310546875 seconds.
6 - T011: 0.00012612342834472656 seconds.
7 - T013: 0.00018835067749023438 seconds.
8 - T015: 0.00018310546875 seconds.
9 - T016: 0.00019407272338867188 seconds.
10 - T017: 0.0001819133758544922 seconds.
11 - T018: 0.0001881122589111328 seconds.
12 - T020: 0.0001709461212158203 seconds.
13 - T021: 0.0001900196075439453 seconds.
14 - T022: 0.0001957416534423828 seconds.
15 - T023: 0.0002129077911376953 seconds.
16 - T024: 0.00020003318786621094 seconds.
17 - T027: 0.00019216537475585938 seconds.
18 - T028: 0.0001800060272216797 seconds.
19 - T031: 0.00011801719665527344 seconds.
20 - T033: 0.00018095970153808594 seconds.
21 - T034: 0.0001857280731201172 seconds.
22 - T035: 0.00013303756713867188 seconds.
23 - T036: 0.00010395050048828125 seconds.
24 -